# Final Project

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB, GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.impute import SimpleImputer

In [3]:
import numpy as np

In [4]:
years = [2018, 2019, 2020, 2021, 2022]
dfs = []
for year in years:
    df = pd.read_csv(f'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_{year}.csv')
    df['year'] = year
    dfs.append(df)
final_df = pd.concat(dfs)

## Data Exploration

Data has been read in, and this is where basic data exploration is taking place.

In [5]:
final_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
0,2018-M020,Brisbane,Hard,32,A,20180101,271,105992,NaN,NaN,...,33.0,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018
1,2018-M020,Brisbane,Hard,32,A,20180101,272,111577,NaN,NaN,...,25.0,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018
2,2018-M020,Brisbane,Hard,32,A,20180101,273,104797,NaN,NaN,...,37.0,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018
3,2018-M020,Brisbane,Hard,32,A,20180101,275,200282,NaN,WC,...,33.0,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018
4,2018-M020,Brisbane,Hard,32,A,20180101,276,111581,NaN,Q,...,28.0,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018


In [6]:
final_df.describe()

,draw_size,tourney_date,match_num,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,best_of,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
count,11823.000000,1.182300e+04,11823.000000,11823.000000,11739.000000,11822.000000,11823.000000,11593.000000,11822.000000,11823.000000,...,11389.000000,11389.000000,11390.000000,11389.000000,11389.000000,11785.000000,11785.000000,11685.000000,11685.000000,11823.000000
mean,60.271336,2.019878e+07,262.492261,120684.644845,187.539484,27.232093,120406.206969,186.652635,27.465970,3.396346,...,33.950654,14.813153,12.544864,4.828431,8.592063,71.396945,1888.729996,101.171673,1197.572871,2019.829654
std,42.579000,1.442071e+04,162.668020,29758.553485,7.091715,4.726342,29747.927903,6.892000,4.690963,0.797280,...,14.571416,6.946332,4.254158,3.294453,4.160238,115.403786,2125.416320,151.615237,1263.425190,1.439202
min,2.000000,2.018010e+07,1.000000,100644.000000,163.000000,14.900000,100644.000000,170.000000,14.500000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2018.000000
25%,32.000000,2.018123e+07,225.500000,105173.000000,183.000000,23.400000,105311.000000,183.000000,23.700000,3.000000,...,23.000000,10.000000,9.000000,2.000000,6.000000,19.000000,729.000000,35.000000,580.000000,2019.000000
50%,32.000000,2.020012e+07,277.000000,106045.000000,188.000000,26.900000,106043.000000,185.000000,27.400000,3.000000,...,32.000000,14.000000,12.000000,4.000000,8.000000,46.000000,1101.000000,66.000000,880.000000,2020.000000
75%,128.000000,2.021072e+07,290.000000,126138.500000,193.000000,30.900000,125802.000000,190.000000,30.800000,3.000000,...,42.000000,19.000000,15.000000,7.000000,11.000000,84.000000,2045.000000,107.000000,1341.000000,2021.000000
max,128.000000,2.022072e+07,1701.000000,211346.000000,211.000000,42.300000,211346.000000,211.000000,42.500000,5.000000,...,171.000000,50.000000,50.000000,27.000000,38.000000,1821.000000,12415.000000,2146.000000,12355.000000,2022.000000


In [7]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11823 entries, 0 to 1925
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          11823 non-null  object 
 1   tourney_name        11823 non-null  object 
 2   surface             11823 non-null  object 
 3   draw_size           11823 non-null  int64  
 4   tourney_level       11823 non-null  object 
 5   tourney_date        11823 non-null  int64  
 6   match_num           11823 non-null  int64  
 7   winner_id           11823 non-null  int64  
 8   winner_seed         4867 non-null   object 
 9   winner_entry        1811 non-null   object 
 10  winner_name         11823 non-null  object 
 11  winner_hand         11821 non-null  object 
 12  winner_ht           11739 non-null  float64
 13  winner_ioc          11823 non-null  object 
 14  winner_age          11822 non-null  float64
 15  loser_id            11823 non-null  int64  
 16  loser

In [8]:
final_df.corr()

,draw_size,tourney_date,match_num,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,best_of,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
draw_size,1.000000,0.041698,0.112829,-0.037526,0.045280,0.047282,-0.048979,0.013873,0.041418,0.755596,...,0.342602,0.327409,0.448059,0.212749,0.300222,-0.200800,0.181435,-0.166581,0.078691,0.039592
tourney_date,0.041698,1.000000,-0.015272,0.300288,-0.006563,-0.066607,0.248213,-0.006959,-0.003250,0.006603,...,-0.018861,-0.047766,-0.028544,0.001134,0.008623,-0.040330,0.074822,-0.012091,0.068816,0.998291
match_num,0.112829,-0.015272,1.000000,-0.034747,0.005644,0.028336,-0.046235,0.010474,0.025298,0.041884,...,0.004372,-0.003197,0.016178,0.023599,0.052056,-0.175221,0.090676,-0.179381,0.086522,-0.016148
winner_id,-0.037526,0.300288,-0.034747,1.000000,-0.021316,-0.622657,0.116998,0.007603,-0.030667,-0.045590,...,-0.039785,-0.035759,-0.044818,-0.009747,-0.009174,0.128582,-0.125625,0.026923,0.010978,0.298869
winner_ht,0.045280,-0.006563,0.005644,-0.021316,1.000000,-0.084912,-0.010156,0.012316,0.005594,0.011252,...,0.068036,0.040621,0.023824,-0.078596,-0.147320,-0.116185,0.165914,-0.050315,0.068209,-0.006549
winner_age,0.047282,-0.066607,0.028336,-0.622657,-0.084912,1.000000,-0.052768,-0.011199,0.020418,0.052369,...,0.034691,0.044193,0.039643,0.017855,0.016681,-0.089737,0.125637,-0.023471,-0.001305,-0.064380
loser_id,-0.048979,0.248213,-0.046235,0.116998,-0.010156,-0.052768,1.000000,0.018537,-0.639903,-0.032702,...,-0.023093,-0.029905,-0.026356,-0.004077,0.004144,0.093223,0.006827,0.182768,-0.071492,0.247373
loser_ht,0.013873,-0.006959,0.010474,0.007603,0.012316,-0.011199,0.018537,1.000000,-0.053890,-0.005864,...,0.142570,0.053187,0.085245,-0.055390,-0.121078,-0.036702,0.045640,-0.094436,0.164181,-0.007241
loser_age,0.041418,-0.003250,0.025298,-0.030667,0.005594,0.020418,-0.639903,-0.053890,1.000000,0.024117,...,0.006707,0.025893,0.008132,-0.003102,-0.014541,-0.075473,-0.009104,-0.125895,0.018819,-0.002985
best_of,0.755596,0.006603,0.041884,-0.045590,0.011252,0.052369,-0.032702,-0.005864,0.024117,1.000000,...,0.478109,0.415928,0.597231,0.293412,0.405308,-0.094466,0.149638,-0.045115,0.005353,0.004588


## Data Cleaning

OK, we have a lot of null values and are going to start with them. 

In [9]:
# Going to start with winner and loser seed
# nan values are probably players that are not seeded at tournaments
final_df['winner_seed'].unique()

array([nan, 8.0, 3.0, 1.0, 7.0, 5.0, 2.0, 4.0, 6.0, 28.0, 24.0, 10.0,
       23.0, 31.0, 30.0, 17.0, 15.0, 26.0, 9.0, 14.0, 21.0, 25.0, 19.0,
       12.0, 13.0, 29.0, 32.0, 18.0, 11.0, 20.0, 22.0, 16.0, 27.0, 33.0,
       '1', '2', '3', '6', '7', '8', '4', '5', '25', '17', '16', '19',
       '31', 'WC', 'Q', '23', '14', '10', '18', '28', '32', '11', '15',
       '20', '26', '27', '24', '9', '13', '12', '33', '29', '30', '21',
       '22', 'LL'], dtype=object)

In [10]:
# nr stands for not ranked
final_df['winner_seed'] = final_df['winner_seed'].replace(np.nan, '0').replace('WC', '0').replace(
'LL', '0').replace('Q', '0')

In [11]:
final_df['loser_seed'] = final_df['loser_seed'].replace(np.nan, '0').replace('WC', '0').replace(
'LL', '0').replace('Q', '0')

In [12]:
# Next we are going to look at winner and loser entry
# I am assuming that nan players are just normal unseeded players and will also be called nr
final_df['winner_entry'].unique()

array([nan, 'WC', 'Q', 'LL', 'SE', 'PR', 'ALT', 'Alt'], dtype=object)

In [13]:
final_df['winner_entry'] = final_df['winner_entry'].replace(np.nan, '0').replace('WC', '0').replace(
'LL', '0').replace('Q', '0')

In [14]:
final_df['loser_entry'] = final_df['loser_entry'].replace(np.nan, '0').replace('WC', '0').replace(
'LL', '0').replace('Q', '0')

In [15]:
# winner_ht, going to replace with mode
# most atp players are about 6' 1"
final_df['winner_ht'].unique()

array([183., 188., 190., 180., 193., 185., 196., 181., 198., 203., 175.,
       172., 178., 170., 208., 173., 191.,  nan, 206., 189., 211., 186.,
       201., 194., 163.])

In [16]:
replace = final_df['winner_ht'].fillna(final_df['winner_ht'].mode()[0], inplace = True)

In [17]:
final_df['winner_ht'].unique()

array([183., 188., 190., 180., 193., 185., 196., 181., 198., 203., 175.,
       172., 178., 170., 208., 173., 191., 206., 189., 211., 186., 201.,
       194., 163.])

In [18]:
replace0 = final_df['loser_ht'].fillna(final_df['winner_ht'].mode()[0], inplace = True)

In [19]:
final_df['winner_ht'].unique()

array([183., 188., 190., 180., 193., 185., 196., 181., 198., 203., 175.,
       172., 178., 170., 208., 173., 191., 206., 189., 211., 186., 201.,
       194., 163.])

In [20]:
# Winner and loser age
# one null in each so just fill with mode 
replace2 = final_df['winner_age'].fillna(final_df['winner_age'].mode()[0], inplace = True)

In [21]:
final_df['winner_age'].isna().sum()

0

In [22]:
replace3 = final_df['loser_age'].fillna(final_df['loser_age'].mode()[0], inplace = True)

In [23]:
final_df['loser_age'].isna().sum()

0

In [24]:
# winner and loser hand
replace4 = final_df['winner_hand'].fillna(final_df['winner_hand'].mode()[0], inplace = True)
replace5 = final_df['loser_hand'].fillna(final_df['loser_hand'].mode()[0], inplace = True)

In [25]:
# dropping data with game stats
# feel it would be inaccurate if we guessed or filled with median/mode/mean
final_df = final_df.dropna()

In [26]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11266 entries, 0 to 1923
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          11266 non-null  object 
 1   tourney_name        11266 non-null  object 
 2   surface             11266 non-null  object 
 3   draw_size           11266 non-null  int64  
 4   tourney_level       11266 non-null  object 
 5   tourney_date        11266 non-null  int64  
 6   match_num           11266 non-null  int64  
 7   winner_id           11266 non-null  int64  
 8   winner_seed         11266 non-null  object 
 9   winner_entry        11266 non-null  object 
 10  winner_name         11266 non-null  object 
 11  winner_hand         11266 non-null  object 
 12  winner_ht           11266 non-null  float64
 13  winner_ioc          11266 non-null  object 
 14  winner_age          11266 non-null  float64
 15  loser_id            11266 non-null  int64  
 16  loser

## Encoding Surface

In [27]:
# Surface
final_df["surface"] = final_df["surface"].astype('category')
final_df.dtypes
final_df["surface_cat"] = final_df["surface"].cat.codes
final_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
0,2018-M020,Brisbane,Hard,32,A,20180101,271,105992,0,0,...,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018,2
1,2018-M020,Brisbane,Hard,32,A,20180101,272,111577,0,0,...,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018,2
2,2018-M020,Brisbane,Hard,32,A,20180101,273,104797,0,0,...,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018,2
3,2018-M020,Brisbane,Hard,32,A,20180101,275,200282,0,0,...,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018,2
4,2018-M020,Brisbane,Hard,32,A,20180101,276,111581,0,0,...,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018,2


## Dropping Columns

In [28]:
# Tournament information
# Country of origin not needed
# winner_entry less useful than seed
# score not needed if we know who already won
# hand not really necessary?
final_df = final_df.drop(columns = ['tourney_id', 'tourney_name', 'tourney_level', 'tourney_date', 'winner_ioc', 'loser_ioc',
                                   'match_num', 'winner_entry', 'loser_entry', 'surface', 'score'])

## Converting DataTypes

We want all columns to be numeric

In [29]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11266 entries, 0 to 1923
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   draw_size           11266 non-null  int64  
 1   winner_id           11266 non-null  int64  
 2   winner_seed         11266 non-null  object 
 3   winner_name         11266 non-null  object 
 4   winner_hand         11266 non-null  object 
 5   winner_ht           11266 non-null  float64
 6   winner_age          11266 non-null  float64
 7   loser_id            11266 non-null  int64  
 8   loser_seed          11266 non-null  object 
 9   loser_name          11266 non-null  object 
 10  loser_hand          11266 non-null  object 
 11  loser_ht            11266 non-null  float64
 12  loser_age           11266 non-null  float64
 13  best_of             11266 non-null  int64  
 14  round               11266 non-null  object 
 15  minutes             11266 non-null  float64
 16  w_ace

In [30]:
# Changing winner and loser seeds to integers
final_df['winner_seed'] = final_df['winner_seed'].astype(int)
final_df['loser_seed'] = final_df['loser_seed'].astype(int)

In [31]:
final_df['round'].loc[final_df['round'] == 'RR'].index

Int64Index([ 266,  267,  268,  269,  270,  271,  272,  273,  275,  277,
            ...
            1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923],
           dtype='int64', length=724)

In [32]:
# Removing the r from round
# will still be interpretable without r (R32 = 32)
# dropping RR, most dont count for points other than atp finals. most are exhibition or matches like laver cup
#getting rid of byes
final_df = final_df.loc[final_df['round'] != 'RR']
final_df = final_df.loc[final_df['round'] != 'B']
final_df = final_df.loc[final_df['round'] != 'BR']

In [33]:
final_df['round'].unique()

array(['R32', 'R16', 'QF', 'SF', 'F', 'R128', 'R64'], dtype=object)

In [34]:
final_df['round'] = final_df['round'].str.replace('R', '')

In [35]:
final_df['round'].unique()

array(['32', '16', 'QF', 'SF', 'F', '128', '64'], dtype=object)

In [36]:
final_df['round'] = final_df['round'].str.replace('QF', '8').replace('SF', '4').replace('F', '2')

In [37]:
final_df['round']

0       32
1       32
2       32
3       32
4       32
        ..
1859    32
1860    32
1861    32
1862    32
1863    32
Name: round, Length: 10541, dtype: object

In [38]:
final_df['round'] = final_df['round'].astype(int)

Need to look at score and name

In [39]:
one_hot = pd.get_dummies(final_df, columns = ['winner_hand'])

In [40]:
one_hot2 = pd.get_dummies(final_df, columns = ['loser_hand'])

In [41]:
final_df.drop(columns = ['winner_hand'], axis = 1)
final_df.merge(one_hot)

,draw_size,winner_id,winner_seed,winner_name,winner_hand,winner_ht,winner_age,loser_id,loser_seed,loser_name,...,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat,winner_hand_L,winner_hand_R,winner_hand_U
0,32,105992,0,Ryan Harrison,R,183.0,25.6,104919,0,Leonardo Mayer,...,4.0,47.0,1010.0,52.0,909.0,2018,2,0,1,0
1,32,111577,0,Jared Donaldson,R,188.0,21.2,111442,0,Jordan Thompson,...,11.0,54.0,890.0,94.0,593.0,2018,2,0,1,0
2,32,104797,0,Denis Istomin,R,188.0,31.3,106000,7,Damir Dzumhur,...,16.0,63.0,809.0,30.0,1391.0,2018,2,0,1,0
3,32,200282,0,Alex De Minaur,R,183.0,18.8,105449,0,Steve Johnson,...,6.0,208.0,245.0,44.0,1055.0,2018,2,0,1,0
4,32,111581,0,Michael Mmoh,R,188.0,19.9,105643,0,Federico Delbonis,...,2.0,175.0,299.0,68.0,755.0,2018,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10536,32,106065,0,Marco Cecchinato,R,185.0,29.8,209890,0,Mili Poljicak,...,24.0,151.0,362.0,537.0,54.0,2022,0,0,1,0
10537,32,106162,0,Franco Agamenone,L,188.0,29.2,111513,0,Laslo Djere,...,10.0,136.0,398.0,77.0,656.0,2022,0,1,0,0
10538,32,144684,5,Alex Molcan,L,178.0,24.6,207213,0,Duje Ajdukovic,...,8.0,38.0,1121.0,276.0,175.0,2022,0,1,0,0
10539,32,106148,0,Roberto Carballes Baena,R,183.0,29.3,105967,0,Henri Laaksonen,...,6.0,86.0,611.0,110.0,518.0,2022,0,0,1,0


In [42]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10541 entries, 0 to 1863
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   draw_size           10541 non-null  int64  
 1   winner_id           10541 non-null  int64  
 2   winner_seed         10541 non-null  int32  
 3   winner_name         10541 non-null  object 
 4   winner_hand         10541 non-null  object 
 5   winner_ht           10541 non-null  float64
 6   winner_age          10541 non-null  float64
 7   loser_id            10541 non-null  int64  
 8   loser_seed          10541 non-null  int32  
 9   loser_name          10541 non-null  object 
 10  loser_hand          10541 non-null  object 
 11  loser_ht            10541 non-null  float64
 12  loser_age           10541 non-null  float64
 13  best_of             10541 non-null  int64  
 14  round               10541 non-null  int32  
 15  minutes             10541 non-null  float64
 16  w_ace

In [43]:
final_df = final_df.reset_index(drop=True)

In [44]:
winners_r = final_df['winner_rank']
losers_r = final_df['loser_rank']
winners_r

0         47.0
1         54.0
2         63.0
3        208.0
4        175.0
         ...  
10536    151.0
10537    136.0
10538     38.0
10539     86.0
10540     59.0
Name: winner_rank, Length: 10541, dtype: float64

In [45]:
winners_r[0]

47.0

In [46]:
final_df

,draw_size,winner_id,winner_seed,winner_name,winner_hand,winner_ht,winner_age,loser_id,loser_seed,loser_name,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
0,32,105992,0,Ryan Harrison,R,183.0,25.6,104919,0,Leonardo Mayer,...,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,2018,2
1,32,111577,0,Jared Donaldson,R,188.0,21.2,111442,0,Jordan Thompson,...,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,2018,2
2,32,104797,0,Denis Istomin,R,188.0,31.3,106000,7,Damir Dzumhur,...,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,2018,2
3,32,200282,0,Alex De Minaur,R,183.0,18.8,105449,0,Steve Johnson,...,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,2018,2
4,32,111581,0,Michael Mmoh,R,188.0,19.9,105643,0,Federico Delbonis,...,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,2018,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10536,32,106065,0,Marco Cecchinato,R,185.0,29.8,209890,0,Mili Poljicak,...,22.0,13.0,18.0,24.0,151.0,362.0,537.0,54.0,2022,0
10537,32,106162,0,Franco Agamenone,L,188.0,29.2,111513,0,Laslo Djere,...,16.0,16.0,5.0,10.0,136.0,398.0,77.0,656.0,2022,0
10538,32,144684,5,Alex Molcan,L,178.0,24.6,207213,0,Duje Ajdukovic,...,18.0,10.0,2.0,8.0,38.0,1121.0,276.0,175.0,2022,0
10539,32,106148,0,Roberto Carballes Baena,R,183.0,29.3,105967,0,Henri Laaksonen,...,17.0,11.0,2.0,6.0,86.0,611.0,110.0,518.0,2022,0


In [47]:
# creating column for favorites for each match based on rankings
favorites = []
for i in list(range(0, 10541)):
    if winners_r[i] > losers_r[i]:             # higher rank is lower value 1 is lowest but best rank
        favorites.append(False)
    elif winners_r[i] < losers_r[i]:
        favorites.append(True)
favorites

[True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 F

In [69]:
len(favorites)

10541

In [48]:
final_df['fav_win'] = favorites

In [49]:
final_df['fav_win'].value_counts()

True     6608
False    3933
Name: fav_win, dtype: int64

In [50]:
X = final_df.drop(columns='fav_win')
y = final_df['fav_win']
# Preparing our hold out data for train test split
X_cols = X.columns
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42, test_size=.25)
X_test, X_hold, y_test, y_hold = train_test_split(X_test, y_test, random_state=42, test_size=.25)

In [51]:
X_train.head()

,draw_size,winner_id,winner_seed,winner_name,winner_hand,winner_ht,winner_age,loser_id,loser_seed,loser_name,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
5034,32,105216,0,Yuichi Sugita,R,173.0,31.0,111200,0,Elias Ymer,...,11.0,10.0,4.0,9.0,129.0,429.0,123.0,455.0,2019,2
8158,128,132283,31,Lorenzo Sonego,R,191.0,25.7,105023,0,Sam Querrey,...,22.0,16.0,7.0,10.0,35.0,1588.0,52.0,1085.0,2021,2
111,32,103852,0,Feliciano Lopez,L,188.0,36.3,126846,0,Aleksandar Vukic,...,24.0,16.0,5.0,7.0,36.0,1295.0,462.0,77.0,2018,2
6451,32,105173,0,Adrian Mannarino,L,183.0,32.9,106281,0,Liam Broady,...,11.0,10.0,6.0,8.0,42.0,1501.0,146.0,499.0,2021,1
9399,32,111578,0,Stefan Kozlov,R,183.0,24.0,105777,0,Grigor Dimitrov,...,17.0,16.0,9.0,13.0,130.0,506.0,33.0,1471.0,2022,2


In [52]:
X_train.groupby('winner_id').mean()

,draw_size,winner_seed,winner_ht,winner_age,loser_id,loser_seed,loser_ht,loser_age,best_of,round,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,surface_cat
winner_id,,,,,,,,,,,,,,,,,,,,,
100644,77.449275,3.239130,198.0,22.895652,122292.152174,3.304348,186.376812,27.575362,3.608696,30.782609,...,14.268116,12.528986,4.898551,9.086957,4.833333,5654.818841,64.050725,1701.659420,2019.760870,1.297101
103333,69.052632,0.368421,208.0,39.642105,116888.631579,1.526316,185.789474,27.494737,3.526316,58.105263,...,20.894737,15.842105,3.105263,4.947368,97.315789,592.105263,89.052632,820.947368,2018.631579,1.263158
103565,128.000000,0.000000,185.0,38.100000,105077.000000,0.000000,188.000000,30.400000,5.000000,128.000000,...,16.000000,14.000000,5.000000,11.000000,158.000000,359.000000,38.000000,1170.000000,2018.000000,1.000000
103819,90.864198,2.358025,185.0,37.511111,109342.469136,4.456790,186.296296,28.404938,3.864198,35.432099,...,16.506173,12.876543,4.876543,8.555556,3.197531,6873.888889,56.679012,1278.777778,2018.765432,1.555556
103852,52.363636,3.454545,188.0,37.703030,123015.060606,2.060606,186.303030,26.430303,3.121212,41.393939,...,17.606061,13.030303,4.484848,7.000000,61.515152,981.333333,97.818182,952.696970,2019.060606,1.242424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208029,34.666667,0.000000,185.0,18.583333,137102.166667,1.333333,187.416667,26.358333,3.000000,24.166667,...,10.416667,10.333333,5.416667,9.250000,131.833333,669.166667,75.166667,1553.250000,2021.583333,0.500000
208103,32.000000,0.000000,185.0,20.480000,137265.000000,1.600000,187.400000,26.120000,3.000000,20.800000,...,16.600000,12.000000,4.600000,7.800000,95.000000,646.800000,63.600000,851.200000,2022.000000,0.800000
208260,128.000000,0.000000,175.0,18.700000,106065.000000,0.000000,185.000000,28.900000,5.000000,128.000000,...,23.000000,23.000000,3.000000,5.000000,716.000000,32.000000,81.000000,911.000000,2021.000000,2.000000


In [73]:
X_trainW = X_train.groupby('winner_id').mean()[['w_df',             
'w_svpt',             
   'w_1stIn',            
   'w_1stWon',           
   'w_2ndWon',           
'w_SvGms',            
  'w_bpSaved',          
  'w_bpFaced',
                                                'w_ace'
    ]]

In [74]:
len(X_trainW)

314

In [75]:
X_trainL = X_train.groupby('loser_id').mean()[['l_ace',              
  'l_df',               
  'l_svpt',             
  'l_1stIn',            
 'l_1stWon',           
  'l_2ndWon',           
  'l_SvGms',            
 'l_bpSaved',          
  'l_bpFaced',
    ]]

In [76]:
WCOPY = X_trainW.copy()
LCOPY = X_trainL.copy()

In [77]:
len(WCOPY)

314

In [78]:
WCOPY.index.names = ['player_id']
LCOPY.index.names= ['player_id']

In [79]:
new_data = WCOPY.merge(LCOPY, how = 'inner', on = 'player_id')
new_data

,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,w_ace,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
player_id,,,,,,,,,,,,,,,,,,
100644,3.702899,75.550725,50.934783,39.644928,12.782609,12.623188,2.949275,4.434783,9.652174,8.341463,6.268293,91.585366,60.439024,42.317073,13.219512,14.829268,4.487805,8.609756
103333,5.368421,97.368421,63.473684,53.578947,18.631579,15.947368,2.631579,3.263158,23.368421,21.862069,4.482759,92.896552,61.448276,50.724138,16.551724,15.275862,2.965517,4.413793
103565,5.000000,86.000000,39.000000,32.000000,28.000000,14.000000,4.000000,5.000000,3.000000,10.500000,6.000000,106.000000,53.500000,37.500000,21.500000,16.000000,10.500000,16.000000
103819,1.703704,76.851852,49.345679,39.222222,16.506173,13.135802,2.703704,3.679012,8.629630,8.277778,2.166667,102.388889,64.722222,47.611111,19.055556,16.277778,4.777778,7.833333
103852,3.606061,82.000000,48.121212,39.575758,18.303030,13.212121,3.333333,4.363636,13.636364,9.921569,4.039216,81.000000,45.490196,33.196078,16.294118,12.509804,4.666667,7.882353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208029,2.000000,63.416667,41.500000,30.666667,12.166667,10.416667,2.666667,4.166667,3.000000,3.526316,4.368421,77.578947,47.000000,30.894737,14.105263,12.368421,4.000000,8.105263
208103,1.600000,73.200000,46.400000,33.800000,13.800000,12.200000,2.000000,4.600000,6.400000,6.625000,4.000000,97.125000,61.750000,42.250000,17.000000,14.875000,6.250000,10.000000
208260,5.000000,133.000000,86.000000,69.000000,26.000000,22.000000,2.000000,3.000000,9.000000,2.000000,2.000000,115.000000,81.000000,46.500000,17.000000,17.000000,6.000000,13.000000


In [80]:
new_data.to_csv('premodel_df')

In [ ]:
for i in new_data['player_id']:
    

In [ ]:
X_trainL.head()

In [68]:
#datastuff = WCOPY.merge(LCOPY, on = 'player_id')

In [69]:
#df.to_csv('datastuff.csv')

## Aggregating columns

Now we can aggregate columns 

## Evaluation

In [ ]:
# Setting our X and y variables
X = final_df.drop(columns='winner_name')
y = final_df['winner_name']
# Preparing our hold out data for train test split
X_cols = X.columns
X, X_hold, y, y_hold = train_test_split(X, y, random_state=42, test_size=.05)
# Regular train test split
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42)

## Logistic Regression 
Compare points and winner name

In [ ]:
# Grouping categorical columns
cat_columns = [c for c in X_train.columns if X_train[c].dtype == 'O']
# Grouping numerical columns
num_columns = [c for c in X_train.columns if X_train[c].dtype in ['int64', 'float64']]
# Combining the numerical and categorical columns 
used_cols = [*cat_columns, *num_columns]

# Instantiating OneHotEncoder(ohe)
cat_transformer = OneHotEncoder(drop='first')
# Instantiating StandardScaler
num_transformer = StandardScaler()

# Instantiating ColumnTransformer
preprocessor = ColumnTransformer([
    ('cats', cat_transformer, cat_columns),
    ('nums', num_transformer, num_columns)
])

In [ ]:
# Function to show train and test accuracy scores
def evaluate(model, Xtr, Xte, ytr, yte):
    train_preds = model.predict(Xtr)
    test_preds = model.predict(Xte)

    train_probas = model.predict_proba(Xtr)[:,1]
    test_probas = model.predict_proba(Xte)[:,1]
    
    plot_confusion_matrix(model, Xte, yte)
    
    plot_roc_curve(model, Xte, yte)
    
    print(f"Train Accuracy: {accuracy_score(y_train, train_preds):.4f}")
    print("*" * 20)
    print(f"Test Accuracy: {accuracy_score(y_test, test_preds):.4f}")

In [ ]:
lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', LogisticRegression())
])

# Fitting model onto train data
lr.fit(X_train[used_cols], y_train)

# Getting score, confusion matrix, and graph
evaluate(lr, X_train[used_cols], X_test[used_cols], y_train, y_test)